# temporal - E04 - multivariate-temporal-dependency.ipynb

### Revised on: 2025-10-28

This example analyzes the multivariate time series of the water current field (mean current velocity, U, and mean incoming current direction, DirU) hindcasted at 0.5058 m below the mean sea level at a point located in 35.916º N, 5.5º W at the Strait of Gibraltar (data provided by [Marine Copernicus System](\href{https://marine.copernicus.eu/)). The hindcast time series has≈27 years duration, with data that spans from 1993/01/01 to 2019/12/31 with a daily temporal cadence. The IBI (Iberian Biscay Irish) Ocean Reanalysis system provides 3D ocean fields (product identifier "IBI_MULTIYEAR_PHY_005_002"). The IBI model numerical core is based on the NEMO v3.6 ocean general circulation model run at 1/12º horizontal resolution. The steps will be the following: 

1. Load environmentaltools packages
2. Read the input data and the dictionaries from previous marginal fit of U and DirU
3. Create the dictionary for the multivariate and temporal dependency analysis and call environmentaltools.temporal.analysis.dependency for fitting the parameters of the VAR
4. Verify the results of the analysis

## 1. Load environmentaltools packages

The following code load the basic functions (read data, analysis and plots) included in environmentaltools

In [ ]:
from environmentaltools.common import read
from environmentaltools.temporal import analysis
from environmentaltools.graphics import plots

## 2. Read the input data and the dictionary from the marginal fit

For examples of reading files go to Ex01, 02 or 03. The following code read the projections of freshwater river discharge from the RCP2.6 of the REMO2009 model in variable called "Qd". The input file is a xlsx file. As usually, some noise is included to ensure that the input variable is continuous and not discrete, which makes more difficult the analysis.

In [ ]:
params = {}
params["U"] = read.rjson("marginalfit/U_norm_nonst_1_trigonometric_8_SLSQP")
params["DirU"] = read.rjson(
        "marginalfit/DirU_weibull_max_nonst_1_trigonometric_8_SLSQP"
    )

First of all, a empty dictionary called params was created. Then, the information from the marginal fits was included (params["U"] and params["DirU"]). From the filename, U timeseries was fit with a non-stationary Gaussian model with a basis period of 1 year using a trigonometric time expansion until order 8. As in general, SLSQP optimize method was chosen. As it can be seen, the information was included in the dictionary called params.

## 3. Create the dictionary for the multivariate and temporal dependency and run the analysis

The next step focus on the creation of the dictionary to the multivariate and temporal dependency. In this case, a VAR model was selected. The variables that go to the analysis are U and DirU and the maximum order to be analysis is 72.

In [ ]:
params["TD"] = {
    "vars": ["U", "DirU"],
    "order": 72,
}

analysis.dependencies(data.loc[:, params["TD"]["vars"]], params)

The parameters of the order with minimum BIC will be saved to a file called "U_DirU_72_VAR.json" in the folder name "dependency". This file contains the parameters of the best fit in the variable "B", the dimension of the fit which is equal to the number of variables (2), the covariance matrix "Q", the BIC of the order (id) selected. 

## 4. Verificate the analysis

The following code will show how to read the results of the dependency analysis. make some plots that show the parameters and relation between modeled and observed variables.

In [ ]:
df_dt = read.rjson("dependency/U_DirU_72_VAR", "td")

The variable "df_dt" is a dictionary with the keys and values given in the previous section. Once the results are read, several plots can be make.


In [ ]:
plots.scatter_error_dependencies(df_dt, params["TD"]["vars"])
plots.heatmap(df_dt["B"], params["TD"], "B")



The first line of the last cell shows a scatter plot of the normalize cumulative distribution function of data and model while the second line shows a heatmap with the parameters. In the last case, the covariance parameters can also be shown changing "B" by "Q".

Further information of this analysis can be found in  [[1]](#1) and [[2]](#2).

## References

<a id="1">[1]</a> 
M. Cobos, P. Otiñar, P. Magaña, A. Lira-Loarca, A. Baquerizo (2021). 
MarineTools.temporal (v 1.0.0): A Python package to simulate Earth and environmental timeseries
Submitted to Environmental Modelling & Software.


<a id="2">[2]</a> 
Cobos, M., Otíñar, P., Magaña, P., Baquerizo, A. (2021).
A method to characterize and simulate climate, earth or environmental vector random processes. 
Submitted to  Probabilistic Engineering & Mechanics.